In [1]:
import numpy as np
import pandas as pd
import ast

from gensim import corpora
import gensim
import pyLDAvis.gensim

from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

import time
from tqdm.notebook import tqdm
import warnings

import matplotlib.pyplot as plt


In [2]:
food = pd.read_csv("./data/food_cluster_no_dup.csv", encoding = 'utf-8-sig')


In [3]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2803 entries, 0 to 2802
Columns: 226 entries, SAMPLE_ID to 발행기관
dtypes: float64(84), int64(131), object(11)
memory usage: 4.8+ MB


In [4]:
food = food.drop_duplicates(["food_name"])

In [5]:
food.head()

,SAMPLE_ID,식품코드,DB군,상용제품,food_name,연도,지역 / 제조사,채취시기,식품상세분류,1회제공량,...,트랜스 리놀레산(18:3t)(%),냉산가용성물질(㎎),식염상당량(g),회분(g),폐기율(%),가식부(%),산가용성물질(%),카페인(㎎),성분표출처,발행기관
0,D000117-ZZ-D06,D000117,음식,품목대표,메밀전병,2019,전국(대표),6월,곡류 및 서류,100.0,...,0,0,0,1.336,0,0,0,0,식약처('14) 명절,식품의약품안전처
1,D000123-ZZ-D06,D000123,음식,품목대표,수수부꾸미,2019,전국(대표),6월,곡류 및 서류,100.0,...,0,0,0,1.358,0,0,0,0,식약처('14) 명절,식품의약품안전처
2,D018006-ZZ-AVG,D018006,음식,품목대표,약식,2020,전국(대표),평균,곡류 및 서류,100.0,...,0,0,0,0.900,0,0,0,0,식약처 영양실태조사('20),식품의약품안전처
3,D012703-ZZ-AVG,D012703,음식,외식,멜로우 머쉬룸 오리지널(F),2019,파파존스,평균,음료류,116.0,...,0,0,0,0.000,0,0,0,0,식약처('19),식품의약품안전처
4,D012878-ZZ-AVG,D012878,음식,외식,땡모반,2019,롯데리아,평균,음료류,300.0,...,0,0,0,0.000,0,0,0,0,식약처('19),식품의약품안전처


In [6]:
food.shape

(2742, 226)

In [7]:
# 데이터프레임을 번호로 접근하기 위해 인덱스 초기화
food.reset_index(inplace=True)

In [8]:
documents = []
processed_data = []

# 키워드 학습을 위해 문서 list에 keyword list 추가
for i in range(food.shape[0]):
    documents.append([food.loc[i, 'food_name'] ,food.loc[i, '지역 / 제조사'], food.loc[i, '식품상세분류']])

print(documents)
# ldamodel을 위한 dictionary, corpus 선언, 그리고 coherence 출력을 위한 processed_data 선언
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(text, allow_update=True) for text in documents]
processed_data = [text for text in documents]

# print(dictionary)

# for c in corpus:
#     print(c)

[['메밀전병', '전국(대표)', '곡류 및 서류'], ['수수부꾸미', '전국(대표)', '곡류 및 서류'], ['약식', '전국(대표)', '곡류 및 서류'], ['멜로우 머쉬룸 오리지널(F)', '파파존스', '음료류'], ['땡모반', '롯데리아', '음료류'], ['핫아메리카노(S)', '롯데리아', '음료류'], ['밀크쉐이크(초코)', '롯데리아', '음료류'], ['초콜렛밀크티 + 밀크폼(hot)L', '공차', '음료류'], ['허니 레몬티?(hot)J', '공차', '음료류'], ['얼그레이 아메리카노 (ice)L', '공차', '음료류'], ['카페라떼 (ice)L', '공차', '음료류'], ['딥모카 카페라떼 (ice)L', '공차', '음료류'], ['시나몬 슈가 스틱', '앤티앤스프레즐', '음료류'], ['크림치즈 스틱', '앤티앤스프레즐', '음료류'], ['데일리주스 오렌지', '파리바게뜨', '음료류'], ['주스 딸바 (ICED)', '파리바게뜨', '음료류'], ['슈퍼베리워터주스', '맘스터치', '음료류'], ['스트로베리밀크(홈카페 전용)', '맘스터치', '음료류'], ['스위트망고(홈카페 전용)', '맘스터치', '음료류'], ['망고주스', 'KFC', '음료류'], ['허니딸기라떼', '던킨도너츠', '음료류'], ['모닝콤보 오렌지 주스', '던킨도너츠', '음료류'], ['뚜레쥬르 스트로베리', '뚜레쥬르', '음료류'], ['그린티 라떼', '요거프레소', '음료류'], ['아이스 그린티 라떼', '요거프레소', '음료류'], ['고구마 라떼 ', '요거프레소', '음료류'], ['아이스 고구마라떼', '요거프레소', '음료류'], ['블랙빈 라떼', '요거프레소', '음료류'], ['아이스 블랙빈 라떼', '요거프레소', '음료류'], ['리얼 홍시라떼', '요거프레소', '음료류'], ['순수 딸기 우유', '요거프레소', '음료류'], ['순수 딸기 바나나 우유', '요거프레소', '음료류'], [

In [9]:
# ldamodel 초기화
num_topics = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None


%time model = LdaModel(corpus = corpus, id2word = dictionary, chunksize = chunksize, alpha ="auto", eta="auto",iterations = iterations, num_topics = num_topics, passes = passes, eval_every = eval_every)


Wall time: 10 s


In [ ]:
# ldamodel coherence, perplexity 출력
cm = CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()
print("Cpherence",coherence)
print('\nPerplexity: ', model.log_perplexity(corpus))

In [ ]:
# ldamodel passes 수 구하기
coherences=[]
perplexities=[]
warnings.filterwarnings('ignore')

for i in tqdm(range(100)):
    
    ntopics, nwords = 14, 100
    if i==0:
        p=1
    else:
        p=i
    tic = time.time()
    lda4 = LdaModel(corpus, id2word=dictionary, num_topics=ntopics, passes=p)
    print('epoch',p,time.time() - tic)
    # tfidf, corpus 무슨 차이?
    # lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=ntopics, iterations=200000)

    cm = CoherenceModel(model=lda4, corpus=corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    print("Cpherence",coherence)
    coherences.append(coherence)
    print('Perplexity: ', lda4.log_perplexity(corpus),'\n\n')
    perplexities.append(lda4.log_perplexity(corpus))

In [ ]:
# ldamodel passes, coherence 그래프
plt.figure(figsize=(20, 10))
plt.plot(coherences,label='passes coherence relationship', color='green')
plt.xlabel('passes', fontsize=30)
plt.ylabel('coherence', fontsize=30)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

In [ ]:
# ldamodel passes, perplexity 그래프
plt.figure(figsize=(20, 10))
plt.plot(perplexities,label='passes perplexity relationship', color='green')
plt.xlabel('passes', fontsize=30)
plt.ylabel('perplexity', fontsize=30)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

In [ ]:
# ldamodel random state 구하기
coherencesR=[]
perplexitiesR=[]
warnings.filterwarnings('ignore')

for i in tqdm(range(100)):
    nwords = 100
    tic = time.time()
    lda4 = LdaModel(corpus, id2word=dictionary, num_topics=14, passes=1, random_state = i)
    print('random_state',i,time.time() - tic)

    cm = CoherenceModel(model=lda4, corpus=corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    print("Cpherence",coherence)
    coherencesR.append(coherence)
    print('Perplexity: ', lda4.log_perplexity(corpus),'\n\n')
    perplexitiesR.append(lda4.log_perplexity(corpus))

In [ ]:
# ldamodel random_state, coherence 그래프
plt.figure(figsize=(20, 10))
plt.plot(coherencesR,label='random_state coherence relationship', color='green')
plt.xlabel('random_state', fontsize=30)
plt.ylabel('coherence', fontsize=30)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

In [ ]:
# ldamodel random_state, perplexity 그래프
plt.figure(figsize=(20, 10))
plt.plot(perplexitiesR,label='random_state perplexity relationship', color='green')
plt.xlabel('random_state', fontsize=30)
plt.ylabel('perplexity', fontsize=30)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

In [10]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 14, id2word=dictionary, passes=1, random_state = 75)
topics = ldamodel.print_topics()
for topic in topics:

    print(topic)

print('\n Perplexity:', ldamodel.log_perplexity(corpus))

coherence_score_lda = CoherenceModel(model=ldamodel, texts=processed_data, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_score_lda.get_coherence()

print('\n Coherence Score:', coherence_score)

(0, '0.164*"치킨류" + 0.113*"피자나라치킨공주" + 0.085*"맘스터치" + 0.051*"전국(대표)" + 0.024*"음료류" + 0.011*"공차" + 0.010*"면류" + 0.007*"부침류" + 0.007*"튀김류" + 0.007*"파파존스"')
(1, '0.155*"전국(대표)" + 0.071*"비알코리아 (주)배스킨라빈스" + 0.044*"빵류" + 0.035*"음료류" + 0.022*"파리바게뜨" + 0.020*"국&탕류" + 0.020*"롯데리아" + 0.015*"버거류" + 0.012*"볶음류" + 0.011*"무침류"')
(2, '0.168*"던킨도너츠" + 0.095*"전국(대표)" + 0.053*"음료류" + 0.045*"빵류" + 0.027*"무침류" + 0.016*"뚜레쥬르" + 0.013*"공차" + 0.008*"면류" + 0.007*"부침류" + 0.005*"샐러드"')
(3, '0.283*"파리바게뜨" + 0.223*"빵류" + 0.218*"뚜레쥬르" + 0.052*"과자류" + 0.014*"전국(대표)" + 0.006*"맥도날드" + 0.006*"서브웨이" + 0.005*"못난이꽈배기" + 0.004*"파파존스" + 0.003*"음료류"')
(4, '0.107*"파리바게뜨" + 0.068*"전국(대표)" + 0.058*"과자류" + 0.024*"음료류" + 0.015*"샌드위치류" + 0.013*"조림류" + 0.011*"빵류" + 0.010*"요거프레소" + 0.009*"공차" + 0.008*"뚜레쥬르"')
(5, '0.083*"전국(대표)" + 0.050*"빵류" + 0.024*"버거류" + 0.023*"파리바게뜨" + 0.022*"버거킹" + 0.015*"튀김류" + 0.013*"국&탕류" + 0.013*"요거프레소" + 0.009*"던킨도너츠" + 0.009*"샌드위치류"')
(6, '0.115*"뚜레쥬르" + 0.055*"과자류" + 0.046*"전국(대표)" + 0.023*"샌드위치류" + 0.01

In [11]:
# 클러스터 시각화
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [12]:
# topictable dataframe 생성 함수
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4)]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [13]:
# 모든 음식의 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중 데이터프레임 생성
topictable = make_topictable_per_doc(ldamodel, corpus)

topictable.columns = ['topic', 'topic_rate']
topictable

,topic,topic_rate
0,1.0,0.7677
1,5.0,0.7678
2,2.0,0.7678
3,9.0,0.7678
4,1.0,0.5100
...,...,...
2737,3.0,0.6913
2738,3.0,0.6907
2739,1.0,0.0714
2740,1.0,0.5411


In [14]:
# 각 클러스터가 음식을 몇개 가지는지 count
topictable['topic'].value_counts()

3.0     534
1.0     418
7.0     283
2.0     282
12.0    270
5.0     153
0.0     131
13.0    117
11.0    109
4.0      95
9.0      93
8.0      90
10.0     86
6.0      81
Name: topic, dtype: int64

In [15]:
# 기존 음식 데이터에 topic 데이터프레임 추가
food_topic = pd.concat([food, topictable], axis = 1)

In [16]:
food_topic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2742 entries, 0 to 2741
Columns: 229 entries, index to topic_rate
dtypes: float64(86), int64(132), object(11)
memory usage: 4.8+ MB


In [17]:
food_topic.head()

,index,SAMPLE_ID,식품코드,DB군,상용제품,food_name,연도,지역 / 제조사,채취시기,식품상세분류,...,식염상당량(g),회분(g),폐기율(%),가식부(%),산가용성물질(%),카페인(㎎),성분표출처,발행기관,topic,topic_rate
0,0,D000117-ZZ-D06,D000117,음식,품목대표,메밀전병,2019,전국(대표),6월,곡류 및 서류,...,0,1.336,0,0,0,0,식약처('14) 명절,식품의약품안전처,1.0,0.7677
1,1,D000123-ZZ-D06,D000123,음식,품목대표,수수부꾸미,2019,전국(대표),6월,곡류 및 서류,...,0,1.358,0,0,0,0,식약처('14) 명절,식품의약품안전처,5.0,0.7678
2,2,D018006-ZZ-AVG,D018006,음식,품목대표,약식,2020,전국(대표),평균,곡류 및 서류,...,0,0.900,0,0,0,0,식약처 영양실태조사('20),식품의약품안전처,2.0,0.7678
3,3,D012703-ZZ-AVG,D012703,음식,외식,멜로우 머쉬룸 오리지널(F),2019,파파존스,평균,음료류,...,0,0.000,0,0,0,0,식약처('19),식품의약품안전처,9.0,0.7678
4,4,D012878-ZZ-AVG,D012878,음식,외식,땡모반,2019,롯데리아,평균,음료류,...,0,0.000,0,0,0,0,식약처('19),식품의약품안전처,1.0,0.5100


In [18]:
# 키워드 입력
documents_input = []
keyword_input = input().split()

documents_input.append(keyword_input)
corpus_input = [dictionary.doc2bow(text) for text in documents_input]
topictable_input = make_topictable_per_doc(ldamodel, corpus_input)
topictable_input.columns = ['topic', 'topic_rate']
# 입력 키워드에 대한 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중 출력
topictable_input

찜류


,topic,topic_rate
0,1.0,0.5357


In [20]:
food_topic.to_csv("./data/food_topic.csv", mode='w', encoding ='utf-8-sig')

In [23]:
youtube = pd.read_csv("./data/youtube_2.csv", encoding = 'utf-8')

In [24]:
youtube.head()

,Column1,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,category
0,1656,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],1966-08-12,UCRuSxVu4iqTK5kCh90ntAgA,총몇명,0,1966-08-12,총몇명|재밌는 만화|부락토스|루시퍼|총몇명 프리퀄|총몇명 스토리,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",Film & Animation
1,1891,Q9t9fl3vHl0,불사신들을 너무 많이 생산해버린 미국 제약 회사,1966-09-09,UCxlv4aOnrRTXMRSL8bVJqEw,B Man 삐맨,0,1966-09-11,더보이즈|더보이즈 시즌1|더보이즈 시즌2|미드 더보이즈|미드 더보이즈 시즌2|미드 ...,504063,4712,58,391,https://i.ytimg.com/vi/Q9t9fl3vHl0/default.jpg,False,False,여러분 안녕하세요 삐맨입니다.오늘은 '더 보이즈' 시즌 2에 대해 알아보려 합니다....,Film & Animation
2,1890,ny5ZVoC-pp4,"승찬, 우정을 느끼다 [총몇명 프리퀄]",1966-09-09,UCRuSxVu4iqTK5kCh90ntAgA,총몇명,0,1966-09-11,총몇명|유승찬|민모리|ㅋㅋㅋ|웃긴|부락토스,348169,38486,684,4607,https://i.ytimg.com/vi/ny5ZVoC-pp4/default.jpg,False,False,"오늘도 역시 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",Film & Animation
3,1889,sIzWUz6OnG8,"묵직하고, 치밀하며, 몰입도 극강인 갓띵작 스릴러 (결말포함)",1966-09-04,UCOXAccjfcIvd-uRQgK2Fsrw,늘보Movie,0,1966-09-10,영화리뷰|늘보무비|인생영화|명작리뷰|프리즈너스,432102,4503,128,293,https://i.ytimg.com/vi/sIzWUz6OnG8/default.jpg,False,False,"프리즈너스 (Prisoners, 613)영화 프리즈너스 소개영상입니다 :)#늘보무비...",Film & Animation
4,1888,c4l8OGyj-lg,모두가 솔직하면 무서운 세상이 펼쳐진다 ㅋㅋㅋㅋ,1966-09-05,UCb_GEWOGkCTXawWZKDtALwg,9bul,0,1966-09-10,구불|거짓말|거짓말의 발명|병맛영화|영화추천,57541,9677,184,1301,https://i.ytimg.com/vi/c4l8OGyj-lg/default.jpg,False,False,메일주소 9bul.contact@gmail.com 공식 페이스북 페이지https:/...,Film & Animation


In [25]:
topic_result = topictable_input['topic'][0]
youtube_cluster = youtube[youtube['categoryId'] == topic_result]

In [26]:
youtube_cluster.head()

,Column1,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,category
346,5988,l6T9xIeZTds,66 Annual Shareholder Meeting and Battery Day,1966-09-23,UC5WjFrtBdufl6CZojX3D8dQ,Tesla,1,1966-09-23,tesla|model s|model x|model 3|powerwall|electr...,1873913,0,0,1849,https://i.ytimg.com/vi/l6T9xIeZTds/default.jpg,False,True,Battery Day Q&A: please submit your questions ...,Autos & Vehicles
347,5899,I4Z0_AygkJg,오가나 입니다.,1966-08-14,UC8av1CNslnPQS3N08rkzzhQ,오프라이드oh-pride,1,1966-08-06,[None],875002,2874,10483,3572,https://i.ytimg.com/vi/I4Z0_AygkJg/default.jpg,False,False,NaN,Autos & Vehicles
348,5903,KO6VgINRUlo,"2열에 누워서 가자! 4세대 카니발 신차 리뷰, 7인승 or 9인승? KIa New...",1966-08-18,UCrXkDAx7w0EbfnyM7yNzmpQ,Motorian모터리언,1,1966-08-21,기아|카니발|신형 카니발|4세대 카니발|Kia|Carnival|New Carnival,185797,1363,93,211,https://i.ytimg.com/vi/KO6VgINRUlo/default.jpg,False,False,4세대 신형 카니발이 공식 출시됐습니다. 신규 플랫폼을 적용하고 차체가 조금 더 커...,Autos & Vehicles
349,5904,gFTTzi3d_Eo,역대급 카니발의 등장! 신형 카니발 7인승과 9인승 살펴보고 왔습니다,1966-08-18,UCp0B9n0YYC8E8bJmS5i4oqw,Motorgraph 모터그래프,1,1966-08-21,모터그래프|motorgraph|자동차|신차|국산차|수입차|전승용|김상영|신승영|강병...,130492,783,93,63,https://i.ytimg.com/vi/gFTTzi3d_Eo/default.jpg,False,False,#기아차 #카니발풀체인지 #카니발실내* 제원표에 오류가 있습니다. 가솔린 모델은 I...,Autos & Vehicles
350,5905,r0SQf5cafc,죄송합니다,1966-08-14,UC8av1CNslnPQS3N08rkzzhQ,오프라이드oh-pride,1,1966-08-21,[None],111261,1812,14972,6285,https://i.ytimg.com/vi/r0SQf5cafc/default.jpg,False,False,NaN,Autos & Vehicles


In [27]:
import pickle
import joblib

In [29]:
save_pkl_1 = pickle.dumps(ldamodel)
save_pkl_2 = pickle.dumps(dictionary)

In [30]:
pkl_1 = pickle.loads(save_pkl_1)
pkl_2 = pickle.loads(save_pkl_2)

In [32]:
joblib.dump(pkl_1,'ldamodel')
joblib.dump(pkl_2, 'dictionary')

['dictionary']